# Accelerated Parameter Mapping of Multiple-Echo Gradient-Echo Data Using Model-Based Iterative Reconstruction

*M. Zimmermann, Z. Abbas, K. Dzieciol and N. J. Shah, "Accelerated Parameter Mapping of Multiple-Echo Gradient-Echo Data Using Model-Based Iterative Reconstruction," in IEEE Transactions on Medical Imaging, vol. 37, no. 2, pp. 626-637, Feb. 2018, doi: 10.1109/TMI.2017.2771504.*

# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import sys

sys.path.insert(0, "../src")

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import h5py
import ismrmrd
import matplotlib.pyplot as plt
import torch

from juart.conopt.functional.fourier import fourier_transform_adjoint

# from juart.preproc.aux import process_siemens_file
from juart.preproc.data import KSpaceData, get_shape
from juart.preproc.trajectory import KSpaceTrajectory

torch.set_num_threads(1)

In [ ]:
fname = "7T1026"

In [ ]:
save = True
pulseq = False

In [ ]:
full_session_dir = "/home/projects/qrage/sessions/%s" % fname
h5_ismrmrd_fname = "ismrmrd/mz_me_mpnrage3d.h5"
h5_preproc_fname = "preproc/mz_me_mpnrage3d_grappa_pytorch.h5"

In [ ]:
full_h5_ismrmrd_fname = os.path.join(full_session_dir, h5_ismrmrd_fname)
full_h5_preproc_fname = os.path.join(full_session_dir, h5_preproc_fname)

In [ ]:
print(full_h5_ismrmrd_fname)
print(full_h5_preproc_fname)

# Load Data

In [ ]:
print("(1/2) Loading rawdata ...")

In [ ]:
dataset = ismrmrd.Dataset(
    full_h5_ismrmrd_fname,
    dataset_name="dataset",
    create_if_needed=False,
)

In [ ]:
NCha, NCol, NLin, NPar, NSli, NSet, NEco = get_shape(dataset)

In [ ]:
# For now, manually set this to 160
NPar = 160

In [ ]:
print(NCha, NCol, NLin, NPar, NSli, NSet, NEco)

In [ ]:
NLin_retro = 8
NPar_post, NSli_post = NSli, NPar
NCha_comp, ISet_comp, IEco_comp = 8, slice(NSet - 1, NSet), slice(0, 1)
NImx, NImy, ISet_coil, IEco_coil = 256, 256, slice(15, 19), slice(0, 1)
NAcl, NUsf = 32, 2

In [ ]:
kdata = KSpaceData((NCha, NCol, NLin, NPar, NSli, NSet, NEco))

In [ ]:
kdata.read_data(dataset, is_pulseq=True)

In [ ]:
# Undersample along line dimension
kdata.index_data([slice(None), slice(None), slice(NLin_retro), ...])

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

# GRAPPA

In [ ]:
kdata.apply_mask(NUsf, NAcl, is_pulseq=True)

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

In [ ]:
kdata.reconstruct_partitions()

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

# Coil compression

In [ ]:
# Swap Par and Sli Partition
kdata.swapaxes(3, 4)

In [ ]:
comp_matrix = kdata.compression_matrix(NCha_comp, ISet_comp, IEco_comp)

In [ ]:
kdata.compress_data(comp_matrix)

In [ ]:
# Swap Col and Lin dimension
kdata.swapaxes(1, 2)

In [ ]:
NCha, NLin, NCol, NPar, NSli, NSet, NEco = kdata.kdata_shape

In [ ]:
print("(4/4) Creating output file ....")

if save:
    with h5py.File(full_h5_preproc_fname, "w", libver="latest") as out_file:
        out_file.create_dataset(
            "d", data=kdata.kdata.numpy(), chunks=(NCha, NLin, NCol, 1, 1, 1, 1)
        )

In [ ]:
plt.figure()
plt.subplot(3, 2, 1)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 0, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 2)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 0, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 3)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 1, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 4)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 1, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 5)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 2, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 6)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 2, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")

In [ ]:
vmax = 5e-4

In [ ]:
plt.figure()
plt.subplot(3, 2, 1)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 0, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 2)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 0, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 3)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 1, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 4)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 1, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 5)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 2, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 6)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 2, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")

# Gradient Delay Correction (RING)

In [ ]:
ktraj = KSpaceTrajectory(NLin, NCol, NSet, NEco)

In [ ]:
S_corr = ktraj.estimate_gradient_delay(kdata.kdata)

In [ ]:
S_corr

In [ ]:
ktraj_corr = ktraj.correct_kspace_trajectory(S_corr)

In [ ]:
if save:
    with h5py.File(full_h5_preproc_fname, "r+", libver="latest") as h5_preproc_file:
        dset = h5_preproc_file.create_dataset(
            "k", data=ktraj_corr.numpy(), chunks=(2, NLin_retro, NCol, 1, 1, 1, 1)
        )

# Coil Sensitivity Estimation (SAKE + ESPIRiT)

In [ ]:
sensmaps = kdata.get_sensmaps(ktraj_corr, (NCha, NImx, NImy, 1, NSli, 1, 1))

In [ ]:
if save:
    with h5py.File(full_h5_preproc_fname, "r+", libver="latest") as h5_preproc_file:
        dset = h5_preproc_file.create_dataset(
            "C",
            data=sensmaps.numpy(),
            chunks=(NCha_comp, NImx, NImy, NPar_post, NSli_post, 1, 1),
        )

In [ ]:
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(torch.abs(sensmaps[0, :, :, 0, 80, 0, 0]).numpy())
plt.subplot(1, 3, 2)
plt.imshow(torch.abs(sensmaps[0, :, 128, 0, :, 0, 0]).T.numpy())
plt.subplot(1, 3, 3)
plt.imshow(torch.abs(sensmaps[0, 128, :, 0, :, 0, 0]).T.numpy())

In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(torch.abs(sensmaps[0, :, 128, 0, :, 0, 0].T).numpy(), vmin=0, vmax=1)
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(torch.abs(sensmaps[-1, :, 128, 0, :, 0, 0].T).numpy(), vmin=0, vmax=1)
plt.axis("off")